__Chapter 14 - Going Deeper – The Mechanics of TensorFlow__

1. [TensorFlow ranks and tensors](#TensorFlow-ranks-and-tensors)
    1. [How to get the rank and shape of a tensor](#How-to-get-the-rank-and-shape-of-a-tensor)
1. [Understanding TensorFlow's computation graphs](#Understanding-TensorFlows-computation-graphs)
1. [](#)
1. [](#)
1. [](#)
1. [](#)
1. [](#)
1. [](#)
1. [](#)
1. [](#)
1. [](#)
1. [](#)
1. [](#)


In [4]:
# Standard libary and settings
import os
import sys
import importlib
import itertools
import warnings; warnings.simplefilter('ignore')
dataPath = os.path.abspath(os.path.join('../../Data'))
modulePath = os.path.abspath(os.path.join('../../CustomModules'))
sys.path.append(modulePath) if modulePath not in sys.path else None
from IPython.core.display import display, HTML; display(HTML("<style>.container { width:95% !important; }</style>"))


# Data extensions and settings
import numpy as np
np.set_printoptions(threshold = np.inf, suppress = True)
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '{:,.6f}'.format


# Modeling extensions
import sklearn.base as base
import sklearn.cluster as cluster
import sklearn.datasets as datasets
import sklearn.decomposition as decomposition
import sklearn.discriminant_analysis as discriminant_analysis
import sklearn.ensemble as ensemble
import sklearn.feature_extraction as feature_extraction
import sklearn.feature_selection as feature_selection
import sklearn.linear_model as linear_model
import sklearn.metrics as metrics
import sklearn.model_selection as model_selection
import sklearn.neighbors as neighbors
import sklearn.pipeline as pipeline
import sklearn.preprocessing as preprocessing
import sklearn.svm as svm
import sklearn.tree as tree
import sklearn.utils as utils


# Visualization extensions and settings
import seaborn as sns
import matplotlib.pyplot as plt


# Custom extensions and settings
from quickplot import qp, qpUtil, qpStyle
from mlTools import powerGridSearch
sns.set(rc = qpStyle.rcGrey)


# Magic functions
%matplotlib inline


<a id = 'TensorFlow-ranks-and-tensors'></a>

# TensorFlow ranks and tensors

The TensorFlow library allows users to define graphs which perform operations and functions over tensors. Tensors are a generalizable mathematical for multidimensional arrays holding data values. The dimensionality of a tensor is generally referred to as its rank. Up to this point, we have worked with tensors of rank zero to two. A scalar is a tensor of rank 0, a vector is a tensor of rank 1, and a matrix is a tensor of rank 2. Tensor notation can be generalized to higher dimensions.

<a id = 'How-to-get-the-rank-and-shape-of-a-tensor'></a>

## How to get the rank and shape of a tensor

TensorFlow has several built in functions which easily facilitate retrieval of a Tensor's rank and shape.

In [9]:
import tensorflow as tf

g = tf.Graph()

# defint the computation graph
with g.as_default():
    # define tensors as t1, t2, t3
    t1 = tf.constant(np.pi)
    t2 = tf.constant([1,2,3,4])
    t3 = tf.constant([[1,2],[3,4]])
    
    # get the ranks
    r1 = tf.rank(t1)
    r2 = tf.rank(t2)
    r3 = tf.rank(t3)
    
    # get the shapes
    s1 = t1.get_shape()
    s2 = t2.get_shape()
    s3 = t3.get_shape()
    print('Shapes: {0}, {1}, {2}'.format(s1,s2,s3))
    
with tf.Session(graph = g) as sess:
    print('Ranks: {0}, {1}, {2}'.format(r1.eval(), r2.eval(), r3.eval()))

Shapes: (), (4,), (2, 2)
Ranks: 0, 1, 2


> Remarks - The rank of the t1 tensor is zero since it's just a scalar. The rank of the t2 vector is 1, and since it has four elements its shape is a one element tupe of (4,). The last shape t3 is a 2 by 2 matrix, and therfore has a rank of 2 and its shape is (2,2).

<a id = 'Understanding-TensorFlows-computation-graphs'></a>

## Understanding TensorFlow's computation graphs

At its core, TensorFlow relies on building a computation graph. This graph is used to derive relationships between tensors from the input all the way through to the output. As an example, if we have rank 0 tensor (a scalar) and tensors a, b and c, and we want to evaluate $z = 2 \times (a-b) + c$. The graph can be described as follows: Tensors a and b feed into $r_1$, which is equal to $a-b$. The resulting $r_1$ is referred to as an intermediate result tensor. $r_1 = a-b$ feeds into another intermediate graph $r_2$, which is equal to $2 \times r_1$. The intermediate result tensor $r_2$, along with c, are fed into the final result tensor $z$, which is equal to $r_2 + c$. This series of operations can be reimagined as a network of nodes, where each node represents an operation where a function is applied to one or more input tensors, and yields zero or more output tensors.

TensorFlow needs to be able to build this computational graph. The individual steps for building a typical graph are:

1. Instantiate a new, empty computation graph
2. Add nodes (tensors and operations) to the graph
3. Execute the graph by:
    1. Start a new session
    2. Initialize the variables in the graph
    3. Run the computation graph in this session

In [10]:
# computation graph for the equation above

g = tf.Graph()

with g.as_default():
    a = tf.constant(1, name = 'a')
    b = tf.constant(2, name = 'b')
    c = tf.constant(3, name = 'c')
    
    z = 2 * (a - b) + c


In [ ]:
In the code above, 'with g.asdefault()' adds nodes described within to the graph

<a id = ''></a>

# A

<a id = ''></a>

# A

<a id = ''></a>

# A

<a id = ''></a>

# A

<a id = ''></a>

# A

<a id = ''></a>

# A

<a id = ''></a>

# A

<a id = ''></a>

# A

<a id = ''></a>

# A

<a id = ''></a>

# A

<a id = ''></a>

# A

<a id = ''></a>

# A